---------------

# **<font style="color:Black">Create OCR by PyTorch</font>**
-------------------
-----------------

## **<font style="color:blue">Installation and import libraries</font>**
-------------------

In [5]:
!pip install  pillow 
!pip install opencv-python

In [14]:
import os
import sys
import shutil
import random
from PIL import Image, ImageDraw, ImageFont, ImageEnhance, ImageFilter
import numpy as np
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

## **<font style="color:blue">Hyperparameters</font>**
-------------------

In [8]:
OUTPUT_DIR = os.path.join('/kaggle','working','synthetic_data','images')
MODEL_DIR = os.path.join('/kaggle','working','model_dir')
LABELS_FILE = os.path.join('/kaggle','working','synthetic_data','labels.txt')
NUM_SAMPLES = 1280  # Number of images generated
IMG_WIDTH = 128
IMG_HEIGHT = 32
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001
CHARSET = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-*"  # Group characters
MAX_TEXT_LENGTH = 10  # Maximum length of text in an image
FONT_DIR = os.path.join('/kaggle','input','google-fonts','GoogleFontScripts') # Folder with TrueType fonts (.ttf)
BACKGROUND_DIR = os.path.join('/kaggle','working','backgrounds')  # New folder for background (optional)
NUMBER_BACKGROUND_IMAGE = 20

## **<font style="color:blue">Utils support functions</font>**
-------------------

### **<font style="color:green">Create output folders</font>**

In [9]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(BACKGROUND_DIR, exist_ok=True)

# Creates a new file
with open(LABELS_FILE, 'w') as fp:
    pass

### **<font style="color:green">Load the font list (add the paths to the .ttf files to the "fonts" folder)</font>**

In [10]:
font_files = [
    os.path.join(FONT_DIR, f) for f in os.listdir(FONT_DIR) 
    if f.endswith('.ttf') and os.path.isfile(os.path.join(FONT_DIR, f))
]
if not font_files:
    raise FileNotFoundError("No fonts found in 'fonts' folder. Add .ttf files!")

### **<font style="color:green">Generating a simple gradient background</font>**

In [11]:
def generate_gradient_background(filename, size=(128, 32)):
    img = Image.new('L', size, color=230)  # Lighter gray as a base
    draw = ImageDraw.Draw(img)
    for y in range(size[1]):
        # Soft gradient with low contrast
        color = int(230 - 20 * (y / size[1]))  # From light gray to slightly darker
        draw.line([(0, y), (size[0], y)], fill=color)
    # Background blur
    img = img.filter(ImageFilter.GaussianBlur(radius=2))
    img.save(os.path.join(BACKGROUND_DIR, filename))

### **<font style="color:green">Generate a background with noise (paper texture)</font>**

In [12]:
def generate_paper_texture(filename, size=(128, 32)):
    img = Image.new('L', size, color=220)  # Lighter gray
    noise = np.random.normal(0, 5, size).astype(np.uint8)  # Less noise
    noise_img = Image.fromarray(noise)
    img.paste(noise_img, (0, 0), noise_img)
    # Blur for a softer effect
    img = img.filter(ImageFilter.GaussianBlur(radius=1.5))
    img.save(os.path.join(BACKGROUND_DIR, filename))

### **<font style="color:green">Creating multiple backgrounds</font>**

In [15]:
for i in range(NUMBER_BACKGROUND_IMAGE):
    generate_gradient_background(f"gradient_{i}.png")
    generate_paper_texture(f"paper_{i}.png")

### **<font style="color:green">Load backgrounds (optional, add images to the "backgrounds" folder)</font>**

In [16]:
background_files = (
    [os.path.join(BACKGROUND_DIR, f) for f in os.listdir(BACKGROUND_DIR) 
     if f.endswith(('.png', '.jpg', '.jpeg'))] if os.path.exists(BACKGROUND_DIR) else []
)

### **<font style="color:green">Random text generation function</font>**

In [17]:
def generate_random_text(max_length):
    length = random.randint(1, max_length)
    return ''.join(random.choice(CHARSET) for _ in range(length))

### **<font style="color:green">Functions for adding noise and distortions</font>**

In [18]:
def add_noise_and_distortion(img):
    img_array = np.array(img)
    # Mild Gaussian noise with lower intensity
    if random.random() > 0.5:  # 50% šance
        noise = np.random.normal(0, random.randint(5, 15), img_array.shape).astype(np.uint8)
        img_array = cv2.add(img_array, noise)
    # Subtle perspective distortion
    rows, cols = img_array.shape
    src_points = np.float32([[0, 0], [cols-1, 0], [0, rows-1], [cols-1, rows-1]])
    dst_points = np.float32([
        [random.uniform(0, 3), random.uniform(0, 3)],
        [cols-1-random.uniform(0, 3), random.uniform(0, 3)],
        [random.uniform(0, 3), rows-1-random.uniform(0, 3)],
        [cols-1-random.uniform(0, 3), rows-1-random.uniform(0, 3)]
    ])
    matrix = cv2.getPerspectiveTransform(src_points, dst_points)
    img_array = cv2.warpPerspective(img_array, matrix, (cols, rows))
    return Image.fromarray(img_array)

### **<font style="color:green">Improved image generation feature</font>**

In [19]:
def generate_synthetic_image(text, font_path, img_size=(IMG_WIDTH, IMG_HEIGHT)):
    # Background: We only use generated backgrounds
    if background_files:
        bg_path = random.choice(background_files)
        img = Image.open(bg_path).convert('L').resize(img_size)
    else:
        img = Image.new('L', img_size, color=230)  # Světlejší šedá jako fallback
        draw = ImageDraw.Draw(img)
        for y in range(img_size[1]):
            color = int(230 - 20 * (y / img_size[1]))  # Jemný gradient
            draw.line([(0, y), (img_size[0], y)], fill=color)
        img = img.filter(ImageFilter.GaussianBlur(radius=2))

    draw = ImageDraw.Draw(img)

    # Larger font size for readability
    font_size = random.randint(20, min(IMG_HEIGHT-2, 28))
    font = ImageFont.truetype(font_path, font_size)

    # Text position
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    if text_width > IMG_WIDTH - 10:
        return generate_synthetic_image(text[:MAX_TEXT_LENGTH//2], font_path, img_size)
    x = random.randint(5, max(5, IMG_WIDTH - text_width - 5))
    y = random.randint(5, max(5, IMG_HEIGHT - text_height - 5))

    # Text Highlighting: Darker Color and Soft Outline
    text_color = random.randint(0, 50)  # Darker shades for contrast
    outline_color = 200  # Light gray outline
    for offset_x in [-1, 0, 1]:
        for offset_y in [-1, 0, 1]:
            if offset_x != 0 or offset_y != 0:
                draw.text((x + offset_x, y + offset_y), text, font=font, fill=outline_color)
    draw.text((x, y), text, font=font, fill=text_color)

    # Subtle noise and distortion (applied at lower intensity)
    img = add_noise_and_distortion(img)

    return img

### **<font style="color:green">Generování datasetu</font>**

In [20]:
def create_synthetic_dataset(num_samples):
    labels = []
    for i in range(num_samples):
        text = generate_random_text(MAX_TEXT_LENGTH)
        if not text:
            continue
        font_path = random.choice(font_files)
        img = generate_synthetic_image(text, font_path)
        img_name = f"img_{i:05d}.png"
        img_path = os.path.join(OUTPUT_DIR, img_name)
        img.save(img_path)
        labels.append(f"{img_name} {text}")
        if i % 100 == 0:
            print(f"Generated {i}/{num_samples} images")
    
    if labels:
        with open(LABELS_FILE, 'w') as f:
            f.write("\n".join(labels))
        print(f"Dataset generated! Images saved in '{OUTPUT_DIR}', labels in '{LABELS_FILE}'")
    else:
        print("No labels generated!")


## **<font style="color:blue">Mapping characters to indices and back</font>**
-------------------

In [21]:
char_to_idx = {char: idx + 1 for idx, char in enumerate(CHARSET)}  # 0 is reserved for blank (CTC)
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

## **<font style="color:blue">Custom dataset</font>**
-------------------

In [22]:
class OCRDataset(Dataset):
    def __init__(self, image_dir, labels_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        # Load labels from file (format: "image_name.jpg text")
        self.data = []
        with open(labels_file, 'r') as f:
            for line in f:
                img_name, text = line.strip().split(' ', 1)
                self.data.append((img_name, text))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name, text = self.data[idx]
        img_path = os.path.join(self.image_dir, img_name)
        
        # Load and preprocess the image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
        img = img / 255.0  # Normalization
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)  # Add channel (1, H, W)

        # Convert text to a sequence of indices
        label = [char_to_idx[c] for c in text if c in char_to_idx]
        label_length = len(label)
        label = torch.tensor(label + [0] * (MAX_TEXT_LENGTH - label_length), dtype=torch.int64)
        
        return img, label, label_length

## **<font style="color:blue">Model definition (CNN + RNN + CTC)</font>**
-------------------

In [23]:
class OCRModel(nn.Module):
    def __init__(self, num_chars):
        super(OCRModel, self).__init__()
        
        # Convolutional layers (feature extraction)
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU()
        )
        
        # Conversion to sequential data for RNN
        self.linear = nn.Linear(256 * (IMG_HEIGHT // 4), 512)
        self.rnn = nn.LSTM(512, 128, num_layers=2, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(128 * 2, num_chars + 1)  # +1 for blank (CTC)

    def forward(self, x):
        # CNN part
        x = self.cnn(x)  # (batch, channels, height, width)
        x = x.permute(0, 3, 1, 2)  # (batch, width, channels, height)
        x = x.reshape(x.size(0), x.size(1), -1)  # (batch, width, features)
        x = self.linear(x)  # (batch, width, 512)
        
        # RNN part
        x, _ = self.rnn(x)  # (batch, width, 256)
        x = self.fc(x)  # (batch, width, num_chars + 1)
        return x

## **<font style="color:blue">Training</font>**
-------------------

In [24]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_idx, (imgs, labels, label_lengths) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        label_lengths = label_lengths.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)  # (batch, seq_len, num_chars + 1)
        outputs = outputs.log_softmax(2)  # CTC očekává log softmax

        batch_size = imgs.size(0)  # Dynamická velikost dávky
        seq_length = outputs.size(1)  # Délka sekvence (počet časových kroků)
        input_lengths = torch.full((batch_size,), seq_length, dtype=torch.long).to(device)

        # Diagnostika pro kontrolu
        print(f"Batch size: {batch_size}")
        print(f"Outputs shape: {outputs.shape}")
        print(f"Input lengths: {input_lengths} (size: {input_lengths.size(0)})")
        print(f"Label lengths: {label_lengths} (size: {label_lengths.size(0)})")

        # Kontrola konzistence
        assert input_lengths.size(0) == batch_size, f"input_lengths size {input_lengths.size(0)} != batch_size {batch_size}"
        assert label_lengths.size(0) == batch_size, f"label_lengths size {label_lengths.size(0)} != batch_size {batch_size}"

        loss = criterion(outputs, labels, input_lengths, label_lengths)
        
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Varování: NaN nebo Inf loss na batch {batch_idx}. Přeskakuji...")
            continue
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        
        if batch_idx % 10 == 0:
            print(f"Epoch {epoch + 1}/{EPOCHS}, Batch {batch_idx}, Loss: {loss.item():.4f}")

    return total_loss / len(train_loader)

## **<font style="color:blue">Inference (prediction)</font>**
-------------------

In [25]:
def decode_prediction(output, idx_to_char):
    output = output.argmax(2).cpu().numpy()  # (batch, seq_len)
    texts = []
    for pred in output:
        pred_text = []
        prev = -1
        for idx in pred:
            if idx != 0 and idx != prev:  # Remove blanks and duplicates
                pred_text.append(idx_to_char.get(idx, ''))
            prev = idx
        texts.append(''.join(pred_text))
    return texts

## **<font style="color:blue">Main launch</font>**
-------------------

In [26]:
if __name__ == "__main__":
    if not font_files:
        print("Download some TrueType fonts (.ttf) and place them in the 'fonts' folder!")
    else:
        create_synthetic_dataset(NUM_SAMPLES)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = OCRDataset(image_dir=OUTPUT_DIR, labels_file=LABELS_FILE)
    if len(dataset) == 0:
        print("Dataset is empty! Check labels.txt or image directory.")
    else:
        train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        model = OCRModel(num_chars=len(CHARSET)).to(device)
        criterion = nn.CTCLoss(blank=0, zero_infinity=True)
        optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

        for epoch in range(EPOCHS):
            loss = train_model(model, train_loader, criterion, optimizer, device)
            print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss:.4f}")

        torch.save(model.state_dict(), os.path.join(MODEL_DIR, 'my_ocr_model.pth'))

        model.eval()
        with torch.no_grad():
            for imgs, _, _ in train_loader:
                imgs = imgs.to(device)
                outputs = model(imgs)
                texts = decode_prediction(outputs, idx_to_char)
                print("Predictions:", texts[:5])
                break

Generated 0/1280 images
Generated 100/1280 images
Generated 200/1280 images
Generated 300/1280 images
Generated 400/1280 images
Generated 500/1280 images
Generated 600/1280 images
Generated 700/1280 images
Generated 800/1280 images
Generated 900/1280 images
Generated 1000/1280 images
Generated 1100/1280 images
Generated 1200/1280 images
Dataset generated! Images saved in '/kaggle/working/synthetic_data/images', labels in '/kaggle/working/synthetic_data/labels.txt'
Batch size: 32
Outputs shape: torch.Size([32, 32, 65])
Input lengths: tensor([32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]) (size: 32)
Label lengths: tensor([ 1,  1, 10,  8,  9,  1,  4,  9,  4,  8,  5,  3,  9,  5,  7,  7, 10,  5,
         4, 10,  3,  3,  4,  7,  1,  9,  7,  4,  8,  5,  3,  1]) (size: 32)
Epoch 1/20, Batch 0, Loss: 37.8696
Batch size: 32
Outputs shape: torch.Size([32, 32, 65])
Input lengths: tensor([32, 32, 32, 32, 32, 3

In [27]:
def zip_folder_with_shutil(source_folder, output_path):
    '''Function for zip dir data'''
    shutil.make_archive(output_path, 'zip', source_folder)

In [28]:
zip_folder_with_shutil('/kaggle/working/backgrounds', '/kaggle/working/backgrounds')
zip_folder_with_shutil('/kaggle/working/synthetic_data', '/kaggle/working/synthetic_data')